In [ ]:
import os
import csv

import cv2
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import find_peaks

In [ ]:
def pen_pressure(image):
    # Flatten the 2D image array to a 1D array
    flattened_image = image.flatten()

    # Calculate mean and standard deviation of pixel intensities
    mean_intensity = np.mean(flattened_image)
    std_dev_intensity = np.std(flattened_image)

    return mean_intensity, std_dev_intensity

def horizontal_projection_profile(image):
    # Apply a binary threshold
    _, binary_img = cv2.threshold(image, 127, 255, cv2.THRESH_BINARY_INV)

    # Dilate the image to emphasize text lines
    kernel = np.ones((5,5), np.uint8)
    dilated_img = cv2.dilate(binary_img, kernel, iterations=1)
    #plt.imshow(dilated_img, cmap='gray')

    # Compute the horizontal projection by summing along the columns
    projection = np.sum(dilated_img, axis=1)

    # # For visualization
    # plt.plot(projection)
    # plt.title("Horizontal Projection Profile")
    # plt.xlabel("Row Number")
    # plt.ylabel("Sum of Pixel Values")
    # plt.show()

    return projection

def writing_irregularity(image):
    prominence_threshold=24000
    projection_profile = horizontal_projection_profile(image)
    peaks, properties = find_peaks(projection_profile, prominence=prominence_threshold)
    print(f"Calculating irregularity using {len(peaks)} peaks found in horizontal projection profile.")
    irregularity = np.mean(calculate_valley_sum(projection_profile, peaks))
    return irregularity

def calculate_valley_sum(projection_profile, peaks):
    valley_sums = []
    for i in range(len(peaks) - 1):
        start_index = peaks[i]
        end_index = peaks[i + 1]
        valley_width = (end_index - start_index) // 2
        
        valley_start = start_index + valley_width
        valley_end = end_index - valley_width
        
        valley_sum = np.sum(projection_profile[valley_start:valley_end])
        valley_sums.append(valley_sum)
        
    return valley_sums

In [ ]:
with open('data.csv', mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['File', 'Mean of Pen Pressure', 'Standard Deviation of Pen Pressure', 'Writing Irregularity'])
    for i, filename in enumerate(os.listdir('./images')):
        print(f"({i}) Processing {filename}")
        image = cv2.imread('./images/'+filename, cv2.IMREAD_GRAYSCALE)
        pressure = pen_pressure(image)
        irregularity = writing_irregularity(image)
        #writer.writerow([filename, pressure[0], pressure[1], irregularity])
        print(irregularity)